In [6]:
import pandas as pd 
import numpy as np 
import random
import os
from imblearn.over_sampling import SMOTE

# 시각화 옵션 
from IPython.display import set_matplotlib_formats


In [7]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

class CFG:
    SEED = 42

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(CFG.SEED) # Seed 고정

def get_x_y(df):
    if 'class' in df.columns:
        df_x = df.drop(columns=['id', 'class'])
        df_y = df['class']
        return df_x, df_y
    else:
        df_x = df.drop(columns=['id'])
        return df_x

train_x, train_y = get_x_y(train)
test_x = get_x_y(test)

In [8]:
X_train =  pd.get_dummies(train_x.drop(columns=['father','mother','gender','SNP_01','SNP_02', 'SNP_06', 'SNP_12']))
test_x = pd.get_dummies(test_x.drop(columns=['father','mother','gender','SNP_01','SNP_02', 'SNP_06', 'SNP_12']))
Y_train = train_y

sampler = SMOTE()
sampler.fit(X_train, Y_train)

X_train, Y_train = sampler.fit_resample(X_train, Y_train)


In [9]:
from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

In [10]:
pipe_list = [('model',SVC())]
pip_model = Pipeline(pipe_list)

In [11]:
hyper_parameter = {'model__C':[0.001],
                  'model__kernel':['linear'],
                  'model__class_weight':[None, 'balanced']}

grid_model = GridSearchCV(pip_model, 
                         param_grid=hyper_parameter, 
                         cv=5, n_jobs = -1, scoring='f1')

grid_model.fit(X_train, Y_train)

/Users/yungchannelforyou/opt/anaconda3/envs/dacon1/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:774: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/yungchannelforyou/opt/anaconda3/envs/dacon1/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/yungchannelforyou/opt/anaconda3/envs/dacon1/lib/python3.7/site-packages/sklearn/metrics/_scorer.py", line 221, in __call__
    sample_weight=sample_weight,
  File "/Users/yungchannelforyou/opt/anaconda3/envs/dacon1/lib/python3.7/site-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/Users/yungchannelforyou/opt/anaconda3/envs/dacon1/lib/python3.7/site-packages/sklearn/metrics/_classification.py", line 1131, in f1_score
 

GridSearchCV(cv=5, estimator=Pipeline(steps=[('model', SVC())]), n_jobs=-1,
             param_grid={'model__C': [0.001],
                         'model__class_weight': [None, 'balanced'],
                         'model__kernel': ['linear']},
             scoring='f1')

In [12]:
best_model = grid_model.best_estimator_
best_model

Pipeline(steps=[('model', SVC(C=0.001, kernel='linear'))])

In [13]:
Y_train_pred = best_model.predict(X_train)
Y_test_pred = best_model.predict(test_x)
submit = pd.read_csv('rmSPN1,2,6,12_c0.001SVM.csv')
Y_test = submit["class"]

In [14]:
test_y_pred = pd.DataFrame(Y_test_pred)
test_y_pred= test_y_pred.replace({0:'A',1:'B',2:'C'})
test_y_pred.value_counts()

B    83
A    51
C    41
dtype: int64

In [15]:
print(classification_report(Y_train, Y_train_pred))

              precision    recall  f1-score   support

           A       0.99      1.00      1.00       114
           B       0.96      0.94      0.95       114
           C       0.94      0.96      0.95       114

    accuracy                           0.96       342
   macro avg       0.96      0.96      0.96       342
weighted avg       0.96      0.96      0.96       342

